In [113]:
import pandas as pd
import numpy as np
import os
from datetime import timedelta, date
from ast import literal_eval

'''This script produces a file which contains the book-arrive delta for each transaction.'''

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# load transaction data
trans_df = pd.read_csv('../../../data/cabot_data/sprint_3/trans_s3_raw.csv',
                       parse_dates=['LOOK_DATE']) # transactions with date

In [114]:
# convert strings tot uples
trans_df['PRODUCT'] = [0 if str(x) == 'nan' else literal_eval(x) for x in trans_df['PRODUCT']]

# get look days
trans_df['LOOK_DAY'] = trans_df['LOOK_DATE'].dt.week * 7 - 7 + trans_df['LOOK_DATE'].dt.dayofweek

# get arrival days
get_day = lambda x: x[1] * 7 - 7 + x[2] if x != 0 else 'nan'
trans_df['ARRIVAL_DAY'] = trans_df['PRODUCT'].apply(get_day)

In [ ]:
trans_df['BA_DIFF'] = [ if str(x) == 'nan' else literal_eval(x) for x in trans_df['PRODUCT']]

In [115]:
trans_df

,LOOK_DATE,INTRADAY,PRODUCT,LOOK_DAY,ARRIVAL_DAY
0,2018-01-02,1,"(CD, 32, 4, 2)",1,221
1,2018-01-02,2,"(DD, 25, 4, 2)",1,172
2,2018-01-02,3,"(DD, 25, 4, 2)",1,172
3,2018-01-02,4,"(DKB, 39, 4, 3)",1,270
4,2018-01-02,5,"(DK, 36, 4, 1)",1,249
5,2018-01-02,6,"(DKB, 39, 4, 3)",1,270
6,2018-01-02,7,"(4BV, 36, 4, 3)",1,249
7,2018-01-02,8,"(4BV, 21, 5, 4)",1,145
8,2018-01-02,9,"(DD, 32, 4, 2)",1,221
9,2018-01-02,10,"(CK, 25, 4, 2)",1,172


0        221
1        172
2        172
3        270
4        249
5        270
6        249
7        145
8        221
9        172
10       138
11       249
12       221
13       241
14       249
15       172
16       172
17       nan
18       nan
19       nan
20       nan
21       nan
22       nan
23       nan
24       nan
25       nan
26       nan
27       nan
28       nan
29       nan
        ... 
24189    nan
24190    nan
24191    nan
24192    nan
24193    nan
24194    nan
24195    nan
24196    nan
24197    nan
24198    nan
24199    nan
24200    nan
24201    nan
24202    nan
24203    nan
24204    nan
24205    nan
24206    nan
24207    nan
24208    nan
24209    nan
24210    nan
24211    nan
24212    nan
24213    nan
24214    nan
24215    nan
24216    nan
24217    nan
24218    nan
Name: PRODUCT, Length: 24219, dtype: object

In [45]:
trans_df['PRODUCT']

0         ('CD', 32, 4, 2)
1         ('DD', 25, 4, 2)
2         ('DD', 25, 4, 2)
3        ('DKB', 39, 4, 3)
4         ('DK', 36, 4, 1)
5        ('DKB', 39, 4, 3)
6        ('4BV', 36, 4, 3)
7        ('4BV', 21, 5, 4)
8         ('DD', 32, 4, 2)
9         ('CK', 25, 4, 2)
10        ('CK', 20, 5, 3)
11        ('DK', 36, 4, 1)
12        ('DK', 32, 4, 1)
13        ('CD', 35, 3, 4)
14       ('4BV', 36, 4, 3)
15        ('DD', 25, 4, 2)
16        ('DK', 25, 4, 2)
17                     NaN
18                     NaN
19                     NaN
20                     NaN
21                     NaN
22                     NaN
23                     NaN
24                     NaN
25                     NaN
26                     NaN
27                     NaN
28                     NaN
29                     NaN
               ...        
24189                  NaN
24190                  NaN
24191                  NaN
24192                  NaN
24193                  NaN
24194                  NaN
2

In [ ]:
# add stay len col to df_data
purchases_df['stay_len'] = (purchases_df['DEPART'] - purchases_df['ARRIVAL']).dt.days
# anything greater than 4 is 4
purchases_df['stay_len'] = purchases_df['stay_len'].apply(lambda x: 4 if x > 4 else x)

# initialize some helper vars
df_grouped = purchases_df.groupby(by=['LOOK_DATE']).count()
periods_per_day = df_grouped['DEPART'].max()

look_start = purchases_df['LOOK_DATE'].min()
look_end = purchases_df['LOOK_DATE'].max()

ssn_start = purchases_df['ARRIVAL'].min()
ssn_end = purchases_df['ARRIVAL'].max()